Following file is used on the 23/09/2025 to find the relation between ur tooltip and grasped 

In [1]:
import rtde_receive
rtde_r = rtde_receive.RTDEReceiveInterface("192.168.1.130")
actual_q = rtde_r.getActualQ()


In [2]:
import rospy
from geometry_msgs.msg import PoseStamped
rospy.init_node("listener")



In [3]:
message = rospy.wait_for_message("/estimated_pose", PoseStamped)
TCP = rtde_r.getActualTCPPose()
print(message)
print(TCP)


header: 
  seq: 37
  stamp: 
    secs: 1759233591
    nsecs:  66014766
  frame_id: "world"
pose: 
  position: 
    x: -0.2758321136528645
    y: -0.790883006757483
    z: 0.35490042196754157
  orientation: 
    x: 0.003671979882779881
    y: 0.24219399124692964
    z: 0.9656393345786585
    w: 0.09425627244121106
[-0.27011420672560016, -0.3865763688048404, 0.5045472229259408, 1.5438026896149892, -0.01577707171357289, -0.08006763829638301]


In [7]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import time
def pose_to_matrix(pose):
    """Convert geometry_msgs/Pose to 4x4 homogeneous matrix."""
    t = pose.position
    q = pose.orientation

    trans = np.array([t.x, t.y, t.z])
    rot = R.from_quat([q.x, q.y, q.z, q.w]).as_matrix()

    T = np.eye(4)
    T[:3,:3] = rot
    T[:3, 3] = trans
    return T

def tcp_to_matrix(tcp):
    """Convert UR RTDE TCP pose [x,y,z,rx,ry,rz] to 4x4 homogeneous matrix."""
    trans = np.array(tcp[:3])
    rotvec = np.array(tcp[3:])
    rot = R.from_rotvec(rotvec).as_matrix()

    T = np.eye(4)
    T[:3,:3] = rot
    T[:3, 3] = trans
    return T

def recalibrate():

    #Move arm so camera has good view of gelsight handle
    global rtde_c
    rtde_c.moveL([-0.27012050067604276, -0.38655268021002365, 0.5045680326907392, 1.5437071890880918, -0.01577808098472903, -0.08004113081502745], 0.2, 0.2)
    print("Movement finished")
    time.sleep(1)
    message = rospy.wait_for_message("/estimated_pose", PoseStamped)
    
    TCP = rtde_r.getActualTCPPose()

    T_base_to_gs = pose_to_matrix(message.pose)
    T_base_to_tcp = tcp_to_matrix(TCP)

    # Relative transform
    global T_tcp_to_gs
    T_tcp_to_gs = np.linalg.inv(T_base_to_tcp) @ T_base_to_gs

    # print("T_TCP_to_GelSight:\n", T_tcp_to_gs)


import rtde_control
global rtde_c
# rtde_c = rtde_control.RTDEControlInterface("192.168.1.130")
try:
    rtde_c = rtde_control.RTDEControlInterface("192.168.1.130")
except:
    pass

global T_tcp_to_gs
recalibrate()

print(T_tcp_to_gs)

# saved pose:
# T_TCP_to_GelSight:
#  [[-0.98523342 -0.10773866 -0.13306945  0.01732142]
#  [-0.16860237  0.47520291  0.86357133 -0.06720553]
#  [-0.02980503  0.87325516 -0.48635079  0.36997999]
#  [ 0.          0.          0.          1.        ]]
#GOOD for gelsight frame at tooltip:
# [[-0.98523696 -0.10540039 -0.1349033   0.03575448]
#  [-0.16418031  0.35844612  0.91900011 -0.15641321]
#  [-0.0485074   0.92758134 -0.37045903  0.40072617]
#  [ 0.          0.          0.          1.        ]]

Movement finished
[[-0.98616361 -0.14413327 -0.08189586  0.03223899]
 [-0.13698179  0.43025991  0.89225131 -0.15851984]
 [-0.09336659  0.89112401 -0.4440503   0.39830884]
 [ 0.          0.          0.          1.        ]]


In [8]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def make_homogeneous_from_pos_rpy_xyz(x, y, z, roll, pitch, yaw):
    """Construct 4x4 homogeneous matrix from position + roll-pitch-yaw (radians)."""
    rot = R.from_euler('xyz', [roll, pitch, yaw]).as_matrix()
    T = np.eye(4)
    T[:3, :3] = rot
    T[:3, 3] = [x, y, z]
    return T

def matrix_to_rtde_pose(T):
    """Convert 4x4 homogeneous transform to RTDE pose [x,y,z, rx,ry,rz] where rx,ry,rz is rotvec."""
    trans = T[:3, 3].copy()
    rotm = T[:3, :3].copy()
    r = R.from_matrix(rotm)
    rotvec = r.as_rotvec()   # length-3 vector (axis * angle)
    return np.concatenate([trans, rotvec]).tolist()

# 2) Desired GelSight pose in base frame: [0, -0.70, 0.20, 0, 0, pi]
# x_des, y_des, z_des = -0.09315984, -0.7293086,   0.20
x_des, y_des, z_des = -0.14315984, -0.7893086,   0.05
roll_des, pitch_des, yaw_des = 0.0, 0.0, np.pi  # radians

T_base_to_gs_target = make_homogeneous_from_pos_rpy_xyz(
    x_des, y_des, z_des, roll_des, pitch_des, yaw_des
)


# Safety check
if 'T_tcp_to_gs' not in globals():
    raise RuntimeError("T_tcp_to_gs not defined. Compute it first (T_tcp_to_gs = inv(T_base_to_tcp) @ T_base_to_gs).")

# Invert T_tcp_to_gs
T_tcp_to_gs_inv = np.linalg.inv(T_tcp_to_gs)

# Compute base-to-tcp target
T_base_to_tcp_target = T_base_to_gs_target @ T_tcp_to_gs_inv
print(T_base_to_tcp_target)

rot = R.from_matrix(T_base_to_tcp_target[0:3, 0:3])
print("Move to vec: ", np.hstack((T_base_to_tcp_target[0:3, 3], rot.as_rotvec())))
# Convert to RTDE pose vector [x,y,z,rx]()


[[ 0.98616361  0.13698179  0.09336659 -0.19042717]
 [ 0.14413327 -0.43025991 -0.89112401 -0.50721748]
 [-0.08189586  0.89225131 -0.4440503   0.37094893]
 [ 0.          0.          0.          1.        ]]
Move to vec:  [-0.19042717 -0.50721748  0.37094893  2.02118293  0.19863316  0.00810511]


In [9]:


rtde_c.moveL(np.hstack((T_base_to_tcp_target[0:3, 3], rot.as_rotvec())), 0.1, 0.1)

True

In [11]:
selection_vector = [0,0,1, 0,0,0]  # control force along Z only
wrench = [0.0, 0.0, -1.0, 0.0, 0.0, 0.0]  # want 5 N pushing in -Z
limits = [0.05, 0.05, 0.02, 0.1, 0.1, 0.1]  # allowed deviations

# start force mode (UR script)
rtde_c.forceMode(T_tcp_to_gs, selection_vector, wrench, 2, limits)
# do probing operations...
# exit force mode


n = 4
for i in range (n):
       print(i)
       # 2) Desired GelSight pose in base frame: [0, -0.70, 0.20, 0, 0, pi]
       x_des, y_des, z_des = -0.14315984, -0.7893086,   0.044
       roll_des, pitch_des, yaw_des = 0.4*np.cos(2*np.pi*i/n), 0.4*np.sin(2*np.pi*i/n), np.pi  # radians

       T_base_to_gs_target = make_homogeneous_from_pos_rpy_xyz(
       x_des, y_des, z_des, roll_des, pitch_des, yaw_des
       )

       # Invert T_tcp_to_gs
       T_tcp_to_gs_inv = np.linalg.inv(T_tcp_to_gs)

       # Compute base-to-tcp target
       T_base_to_tcp_target = T_base_to_gs_target @ T_tcp_to_gs_inv

       rot = R.from_matrix(T_base_to_tcp_target[0:3, 0:3])

       #Circle above:
       rtde_c.moveL(np.hstack((T_base_to_tcp_target[0:3, 3], rot.as_rotvec())), 0.4, 0.4)
       
       #Poke:
       rtde_c.moveL(np.hstack((T_base_to_tcp_target[0:3, 3], rot.as_rotvec()))-np.array([0,0,0.018, 0, 0, 0]), 0.1, 0.1)
       
       #Circle above:
       rtde_c.moveL(np.hstack((T_base_to_tcp_target[0:3, 3], rot.as_rotvec())), 0.4, 0.4)


rtde_c.end_force_mode()

TypeError: forceMode(): incompatible function arguments. The following argument types are supported:
    1. (self: rtde_control.RTDEControlInterface, arg0: List[float], arg1: List[int], arg2: List[float], arg3: int, arg4: List[float]) -> bool

Invoked with: <rtde_control.RTDEControlInterface>, array([[-0.98616361, -0.14413327, -0.08189586,  0.03223899],
       [-0.13698179,  0.43025991,  0.89225131, -0.15851984],
       [-0.09336659,  0.89112401, -0.4440503 ,  0.39830884],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), [0, 0, 1, 0, 0, 0], [0.0, 0.0, -1.0, 0.0, 0.0, 0.0], 2, [0.05, 0.05, 0.02, 0.1, 0.1, 0.1]

In [66]:
message_goal = rospy.wait_for_message("/estimated_pose", PoseStamped)
TCP_goal = rtde_r.getActualTCPPose()
message_goal = pose_to_matrix(message_goal.pose)
rot = R.from_matrix(message_goal[0:3, 0:3])
print("Goal vec: ", np.hstack((message_goal[0:3, 3], rot.as_rotvec())))
print(TCP_goal)

Goal vec:  [-0.26543589 -0.79429486  0.35962435 -0.16522748  0.65708237  2.92341798]
[-0.27012050067604276, -0.38655268021002365, 0.5045680326907392, 1.5437071890880918, -0.01577808098472903, -0.08004113081502745]


In [12]:
rtde_c.zeroFtSensor()

while True:
    print(rtde_r.getActualTCPForce())
    time.sleep(0.2)

[0.9077320882939589, 0.4604492651357139, 0.15542287735375604, -0.003846578378888744, 0.002538504085743116, -0.02999279939805756]
[0.33733900279002077, -0.07303131684130484, 0.06167571434790904, -0.010620461312868121, 0.0004296208285361745, -0.010248468284049739]
[0.07271478324380765, 0.3168992494463652, 0.21200577246493435, -0.005027238353144978, 0.00033612708856505056, -0.005798890521947721]
[0.4169848479726022, 0.07308380932168887, -0.1421403667160393, -0.01147648580364388, -0.0028255307611054093, -0.017698953340021077]
[-0.369905568878515, 0.044721799313752954, 0.0559411998226758, -0.007594756465530193, -0.001601138180405073, 0.006420879374109673]
[0.5635420545072376, -0.45659058660321156, -0.044376684644073267, -0.0004230278494683208, 0.0019337337343536508, -0.012842548640750665]
[0.43245668842353147, -0.49417975061010877, 0.3257669356214697, 0.0025743659405998527, -0.002594525046454918, -0.00850025173804844]
[0.21774263521013495, -0.14323860660640475, -0.09062576578596049, -0.0071

KeyboardInterrupt: 

RTDEReceiveInterface boost system Exception: (system:125) Operation canceled
RTDEControlInterface: Could not receive data from robot...
RTDEControlInterface Exception: Operation canceled
RTDEControlInterface: Robot is disconnected, reconnecting...


Reconnecting...
RTDEControlInterface: Successfully reconnected!
Reconnecting...


RTDEControlInterface: Robot is disconnected, reconnecting...


...

RTDEControlInterface: Successfully reconnected!
Reconnecting...


RTDEControlInterface: Robot is disconnected, reconnecting...


RTDEControlInterface: Successfully reconnected!
Reconnecting...


RTDEControlInterface: Robot is disconnected, reconnecting...


: 

In [23]:
import spatialmath as sm
import numpy as np
from scipy.spatial.transform import Rotation as R
T = np.array([[1,0,0,1],
              [0,1,0,2],
              [0,0,1,3],
              [0,0,0,1]])
print(T)

T_se3 = sm.SE3(T)
print(T_se3)

vec = [0.1, 0.2, 0.3, 0.4, 0.5, np.pi]
rot = R.from_rotvec(vec[3:]).as_matrix()
T = np.vstack([np.hstack([rot, np.array([vec[:3]]).T]), np.array([[0,0,0,1]])])
T_vec = sm.SE3(T)
print(T_vec)

[[1 0 0 1]
 [0 1 0 2]
 [0 0 1 3]
 [0 0 0 1]]
   1         0         0         1         
   0         1         0         2         
   0         0         1         3         
   0         0         0         1         

  -0.9668    0.1021    0.2342    0.1       
  -0.02437  -0.9493    0.3133    0.2       
   0.2543    0.2972    0.9203    0.3       
   0         0         0         1         



In [ ]:
import cv2
device = "/dev/v4l/by-id/usb-Arducam_Technology_Co.__Ltd._GelSight_Mini_R0B_2G8V-4TXV_2G8V4TXV-video-index0"
cap = cv2.VideoCapture("/dev/video6", cv2.CAP_DSHOW)

if not cap.isOpened():
    print("Cannot open camera")

In [19]:
import cv2

cap = cv2.VideoCapture("/dev/video6")
if not cap.isOpened():
    print("Cannot open camera")
while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow("GelSight", frame)
    if cv2.waitKey(1) == 27:  # ESC to quit
        break
cap.release()
cv2.destroyAllWindows()